## Preprocessing Data

In [ ]:
import numpy as np
import pandas as pd
import keras

from keras import backend as K
from keras.models import Sequential
from keras.layers import Activation
from keras.layers.core import Dense
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy

%matplotlib inline
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix
import itertools
import matplotlib.pyplot as plt

In [ ]:
general_train = pd.read_csv("./health.csv", names=["age", "health"])
shuffled_general_train = general_train.sample(frac=1)

general_test = pd.read_csv("./health_test.csv", names=["age", "health"])
shuffled_general_test = general_test.sample(frac=1)

In [ ]:
train_labels, train_samples = np.array(shuffled_general_train["health"]), np.array(shuffled_general_train["age"])
test_labels, test_samples = np.array(shuffled_general_test["health"]), np.array(shuffled_general_test["age"])

In [ ]:
scaler = MinMaxScaler(feature_range=(0,1))
scaled_train_samples = scaler.fit_transform((train_samples).reshape(-1,1))

In [ ]:
#print scaled data
for i in scaled_train_samples:
    print(i)

## Simple Sequential Model


In [ ]:
model = Sequential([
    Dense(16, input_shape=(1,), activation='relu'),
    Dense(32, activation='relu'),
    Dense(2, activation='softmax')
])

In [ ]:
model.summary()

In [ ]:
model.compile(Adam(learning_rate=.0003), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(scaled_train_samples, train_labels, validation_split=0.1, batch_size=10, epochs=20, shuffle=True, verbose=2)

## Preprocess Test Data

In [ ]:
scaler = MinMaxScaler(feature_range=(0,1))
scaled_test_samples = scaler.fit_transform((test_samples).reshape(-1,1))

## Predict

In [ ]:
predictions = model.predict(scaled_test_samples, batch_size=10, verbose=0) 

In [ ]:
for i in predictions:
    print(i)

In [ ]:
rounded_predictions = np.argmax(model.predict(scaled_test_samples, batch_size=10, verbose=0), axis=1 )

In [ ]:
for i in rounded_predictions:
    print(i)

## Confusion Matrix

In [ ]:
cm = confusion_matrix(test_labels, rounded_predictions)

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
cm_plot_labels = ['Healthy','Unhealthy']
plot_confusion_matrix(cm, cm_plot_labels, title='Confusion Matrix')